In [34]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported


In [54]:
address = 'Rio de Janeiro, Rio de Janeiro'

geolocator = Nominatim(user_agent = 'rio_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical cooridnate of Rio de Janerio is {},{}.'.format(latitude,longitude))


The geographical cooridnate of Rio de Janerio is -22.9110137,-43.2093727.


In [45]:
CLIENT_ID = 'XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK'
CLIENT_SECRET = 'HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV'
VERSION = '20180605'
LIMIT = 100
print('Your credentials:')
print('CLIENT ID: ' + CLIENT_ID)
print('CLIENT SECRET '+ CLIENT_SECRET)


Your credentials:
CLIENT ID: XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK
CLIENT SECRET HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV


In [56]:
search_query = 'Resturant'
radius = 50000
print(search_quiry+'......OK!')


Resturant......OK!


In [57]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,latitude,longitude,VERSION,search_query,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK&client_secret=HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV&ll=-22.9110137,-43.2093727&v=20180605&query=Resturant&radius=50000&limit=100'

In [58]:
results = requests.get(url).json()


In [59]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress
0,511e61dce4b037cb0faff473,maria maria Resturante,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",v-1599049640,False,-22.900969,-43.177235,"[{'label': 'display', 'lat': -22.9009686764816...",3479,BR,Brasil,[Brasil]
1,53403f9d498eafb4166945e5,Resturante Da Família,"[{'id': '52e81612bcbc57f1066b79f4', 'name': 'B...",v-1599049640,False,-22.757976,-43.433502,"[{'label': 'display', 'lat': -22.7579762452723...",28617,BR,Brasil,[Brasil]


In [60]:
filtered_columns = ['name','categories'] + [col for col in dataframe.columns if col.startswith('location.')]+['id']
dataframe_filtered = dataframe.loc[:,filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list)== 0:
        return None
    else:
        return categories_list[0]['name']
    
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type,axis = 1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,id
0,maria maria Resturante,Brazilian Restaurant,-22.900969,-43.177235,"[{'label': 'display', 'lat': -22.9009686764816...",3479,BR,Brasil,[Brasil],511e61dce4b037cb0faff473
1,Resturante Da Família,Buffet,-22.757976,-43.433502,"[{'label': 'display', 'lat': -22.7579762452723...",28617,BR,Brasil,[Brasil],53403f9d498eafb4166945e5


In [61]:
venues_map = folium.Map(location = [latitude,longitude],zoom_start = 13)

folium.features.CircleMarker(
    [latitude,longitude],
    radius = 10,
    color = 'red',
    popup = 'Condrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6,
).add_to(venues_map)


for lat,lng, label in zip(dataframe_filtered.lat,dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat,lng],
        radius = 5,
        color = 'blue',
        popup = 'label',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6,
    ).add_to(venues_map)
    
venues_map